In [1]:
import os
import sys
import time
import argparse
from pathlib import Path

import numpy as np
import pandas as pd
import mudata as mu
import pyranges as pr

import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    roc_curve,
    precision_recall_curve
)

from typing import Literal


import sys
from pathlib import Path

# Append the parent directory of the current notebook
sys.path.append(str(Path().resolve().parent))  # if utils is in ../utils relative to the notebook
# Or more explicitly:
# sys.path.append(str(Path.cwd().parent))

# Now import the module
from utils.benchmarking_metrics import *

GRN_TOOLS = Literal["scenicplus", "celloracle", "linger"]
SCORE_COL = "score_rank"
TF2GENE_W_COL = "TF2Gene_weight"

def modality_names(grn_tool:GRN_TOOLS, cell_type_col, sample):
    if grn_tool == "scenicplus":
        if "metacell_0" in sample:
            return "scRNA_counts", "scATAC_counts", f"scRNA_counts:{cell_type_col}"
        else:
            return "scRNA_counts", "scATAC_counts", "scRNA_counts:Metacell_Key"
    if grn_tool == "celloracle":
        if "wihtout_atac" in sample:
            return "rna", "", f"rna:{cell_type_col}"
        else:
            return "rna", "atac", f"rna:{cell_type_col}"
    if grn_tool == "linger":
        return "rna", "atac", f"rna:{cell_type_col}"

def get_adata(mudata, grn_tool:GRN_TOOLS):
    if grn_tool == "scenicplus":
        return mudata["scRNA_counts"]
    elif grn_tool == "celloracle" or grn_tool == "linger":
        return mudata["rna"]
    else:
        raise ValueError(f"Unsupported GRN inference tool: {grn_tool}")


def get_mudata(path:str, grn_tool:GRN_TOOLS):
    if grn_tool == "scenicplus" or grn_tool == "celloracle" or grn_tool == "linger":
        mdata = mu.read_h5mu(path)
        return mdata
    else:
        raise ValueError(f"Unsupported GRN inference tool: {grn_tool}")

# GRNs need to have the following columns: "TF", "Gene", "Region"
def preprocess_scenicplus(scplus_mdata):
    # Extract metadata
    direct_df = pd.DataFrame(scplus_mdata.uns['direct_e_regulon_metadata'])
    extended_df = pd.DataFrame(scplus_mdata.uns['extended_e_regulon_metadata'])

    # Combine into one DataFrame
    grn = pd.concat([direct_df, extended_df], ignore_index=True)

    # Filter the relevant columns
    grn_filtered = grn[['Region', 'Gene', 'TF', 'importance_TF2G', 'importance_R2G','regulation', 'rho_TF2G', 'triplet_rank']].copy()

    # Split the 'Region' column into Chromosome, Start, End
    region_split = grn_filtered['Region'].str.extract(r'(chr[\w]+):(\d+)-(\d+)')
    region_split.columns = ['Chromosome', 'Start', 'End']

    # Convert Start and End to integers
    region_split['Start'] = region_split['Start'].astype(int)
    region_split['End'] = region_split['End'].astype(int)

    grn = pd.concat([region_split, grn_filtered], axis=1)

    max_rank = grn["triplet_rank"].max()
    min_rank = grn["triplet_rank"].min()

    grn[SCORE_COL] = (max_rank - grn["triplet_rank"]) / (max_rank - min_rank)

    raw = grn["importance_TF2G"] * grn["rho_TF2G"]
    grn[TF2GENE_W_COL] = np.tanh(3 * raw) 

    return grn

def preprocess_celloracle(mudata):
    """
    Preprocess CellOracle MuData to extract GRN matrix.
    Assumes the GRN is stored in the 'links' attribute of the MuData object.
    """

    grn = pd.DataFrame(mudata.uns['celloracle_links'])

    # Clean the -logp column
    grn["-logp"] = grn["-logp"].replace([np.inf, -np.inf], np.nan)
    grn["-logp"] = grn["-logp"].fillna(0)

    # Optional: coerce -0.0 → 0.0 for cosmetic + numeric sanity
    grn["-logp"] = grn["-logp"].apply(lambda x: 0.0 if x == 0 else x)

    # Apply ranking
    ranks = grn["-logp"].rank(method="average")  # you can also try "dense" or "first"

    # Normalize ranks to [0, 1]
    min_rank, max_rank = ranks.min(), ranks.max()
    if max_rank > min_rank:
        grn[SCORE_COL] = (ranks - min_rank) / (max_rank - min_rank)
    else:
        grn[SCORE_COL] = 0.0


    raw_score = grn["coef_mean"] * grn["-logp"]
    grn[TF2GENE_W_COL] = np.tanh(3 * raw_score)

    grn["Region"] = grn["chromosome"] + ":" + grn["start"].astype(str) + "-" + grn["end"].astype(str)

    grn = grn.rename(columns={
    "source": "TF",
    "target": "Gene",
    "chromosome": "Chromosome",
    "start": "Start",
    "end": "End"
    })

    return grn

def preprocess_linger(mudata):
    """
    Preprocesses a MuData object for Linger-based GRN inference.

    0) Extracts GRN from mudata.uns["grn"]
    1) Parses 'Region' into 'Chromosome', 'Start', 'End'
    2) Computes 'triplet_score' = |tf_re_score * tg_re_score| (min-max normalized)
    3) Computes 'tf2gene_weight' = tanh(3 * tf_tg_score)
    """
    
    # Step 0: Extract the GRN DataFrame
    grn = mudata.uns["grn"]

    # Step 1: Parse 'Region' into 'Chromosome', 'Start', 'End'
    region_split = grn["Region"].str.extract(r"^(chr[^\:]+):(\d+)-(\d+)$")
    grn["Chromosome"] = region_split[0]
    grn["Start"] = region_split[1].astype(int)
    grn["End"] = region_split[2].astype(int)

    # Step 2: Compute 'triplet_score' = |tf_re_score * tg_re_score|, normalized
    if "tf_re_score" not in grn.columns or "tg_re_score" not in grn.columns:
        raise ValueError("Expected columns 'tf_re_score' and 'tg_re_score' not found in GRN.")
    
    # Compute raw score
    grn["raw_score"] = grn["tf_re_score"] * grn["tg_re_score"] * grn["tf_tg_score"]

    # Use absolute values for ranking (if that’s your criterion)
    abs_score = grn["raw_score"].abs()

    # Create rank (method="average" handles ties gracefully)
    ranks = abs_score.rank(method="average")

    # Normalize ranks to [0,1]
    min_rank, max_rank = ranks.min(), ranks.max()
    if max_rank > min_rank:
        grn[SCORE_COL] = (ranks - min_rank) / (max_rank - min_rank)
    else:
        grn[SCORE_COL] = 0.0

    # Step 3: Compute 'tf2gene_weight' = tanh(3 * tf_tg_score)
    if "tf_tg_score" not in grn.columns:
        raise ValueError("Expected column 'tf_tg_score' not found in GRN.")
    
    grn[TF2GENE_W_COL] = np.tanh(3 * grn["tf_tg_score"])

    # fix on mudata
    adata_rna = mudata.mod["rna"]
    adata_atac = mudata.mod["atac"]

    adata_rna.obs_names = adata_rna.obs['barcode'].astype(str)
    adata_atac.obs_names = adata_atac.obs['barcode'].astype(str)
    adata_atac.var_names = adata_atac.var['gene_ids'].astype(str)
    
    return grn


def get_grn_matrix(mudata, grn_tool:GRN_TOOLS):
    if grn_tool == "scenicplus":
        return preprocess_scenicplus(mudata)
    if grn_tool == "celloracle":
        return preprocess_celloracle(mudata)
    if grn_tool == "linger":
        return preprocess_linger(mudata)


def get_benchmark_matrix(tfb_path, prt_path, frc_path, gst_path, tfm_path, c2g_path):
    tfb_matrix = pd.read_csv(tfb_path, sep="\t", header=None)
    tfb_matrix.columns = ["Chromosome", "Start", "End", "TF"]

    prt_matrix = pd.read_csv(prt_path, sep=',')

    frc_matrix = pd.read_csv(frc_path, index_col=0, sep=',')

    gst_matrix = pd.read_csv(gst_path)

    tfm_matrix = df = pd.read_csv(tfm_path, header=None, names=['gene'])

    c2g_matrix = pd.read_csv(c2g_path, sep="\t", header=None, names=["Chromosome", "Start", "End", "Gene", "Sample"], engine="python")


    return tfb_matrix, prt_matrix, frc_matrix, gst_matrix, tfm_matrix, c2g_matrix



/home/andrem/GRN-project/workflow/.snakemake/conda/5218d1ef85cb5f9311f12366d8b1125f_/lib/python3.10/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from argparse import Namespace

args = Namespace()
args.grn_path = "/home/andrem/GRN-project/data/output/scenicplus/results/AML12_DX_metacell_2/scplusmdata.h5mu"
args.grn_tool = "scenicplus"  # or "celloracle", "linger"
args.tfb_golden = "/data/benchmarks/andrem/data/pAML/benchmarks/tf-binding-test/dbs/tf_bind_db.bed"
args.prt_golden = "/data/benchmarks/andrem/data/pAML/benchmarks/tf-actvity/knock_tf_benchmark_data.csv"
args.frc_golden = "/data/benchmarks/andrem/data/pAML/benchmarks/tf-actvity/knock_tf_benchmark_data.csv"
args.gst_golden = "/data/benchmarks/andrem/data/pAML/benchmarks/gsets/merged_network.csv"
args.tfm_golden = "/data/benchmarks/andrem/data/pAML/benchmarks/tfm/filtered_tf_markers.txt"
args.c2g_golden = "/home/andrem/GRN-project/data/input/data/pAML/benchmarks/cre2gene/eqtlcatalogue.bed"
args.celltype_col = "Classified_Celltype"
args.project_name = "AML12_REL_MO_scdart"


In [3]:
tfb_golden, prt_golden, frc_golden, gst_matrix, tfm_matrix, c2g_matrix = get_benchmark_matrix(
    args.tfb_golden,
    args.prt_golden,
    args.frc_golden,
    args.gst_golden,
    args.tfm_golden,
    args.c2g_golden
)

In [3]:
mudata = get_mudata(args.grn_path, args.grn_tool)
grn_inferred = get_grn_matrix(mudata, args.grn_tool)
adata = get_adata(mudata, args.grn_tool)


In [5]:
grn_inferred["Gene"].nunique()

6363

In [5]:
rna_mod_name, atac_mod_name, celltype_col = modality_names(args.grn_tool, args.celltype_col, args.project_name)

In [6]:
omics_r2g = omic_test(
    grn_inferred = grn_inferred,
    mdata = mudata.copy(),
    score_column = SCORE_COL,
    source_column = "Region",
    target_column = "Gene",
    mod_source = atac_mod_name,
    mod_target= rna_mod_name,
    celltype_column= celltype_col,
    step = 0.3
)

 12%|█▏        | 174/1442 [45:03<5:28:21, 15.54s/it] 


KeyboardInterrupt: 